Parts 1 and 2 were done separately and then combined in this file for user convenience

# 1 - Parsing all news' URLs with tag of football club "Spartak" from Sports.ru website 

In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [37]:
# Create a list of links to parse

Start_url='https://www.sports.ru/football/club/spartak/news/' # Start page
URL_List = [] # empty list to store urls
URL_List.append(Start_url) # Appending empty list with the url of Start page

#loop for creating list of URLS
for i in range(2, 21): #Limit of pages shown is 20 on the Sports.Ru website 
    New_Url=Start_url+f'page{i}' # Link format like in this example --> https://www.sports.ru/football/club/zenit/news/page3/
    URL_List.append(New_Url) # Appending existing list with url of new page i

URL_List #show results

['https://www.sports.ru/football/club/spartak/news/',
 'https://www.sports.ru/football/club/spartak/news/page2',
 'https://www.sports.ru/football/club/spartak/news/page3',
 'https://www.sports.ru/football/club/spartak/news/page4',
 'https://www.sports.ru/football/club/spartak/news/page5',
 'https://www.sports.ru/football/club/spartak/news/page6',
 'https://www.sports.ru/football/club/spartak/news/page7',
 'https://www.sports.ru/football/club/spartak/news/page8',
 'https://www.sports.ru/football/club/spartak/news/page9',
 'https://www.sports.ru/football/club/spartak/news/page10',
 'https://www.sports.ru/football/club/spartak/news/page11',
 'https://www.sports.ru/football/club/spartak/news/page12',
 'https://www.sports.ru/football/club/spartak/news/page13',
 'https://www.sports.ru/football/club/spartak/news/page14',
 'https://www.sports.ru/football/club/spartak/news/page15',
 'https://www.sports.ru/football/club/spartak/news/page16',
 'https://www.sports.ru/football/club/spartak/news/pag

In [38]:
# Iterate over each page URL
for url in URL_List:
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all <p> tags with class "one_news" and the specified data attributes
        news_elements = soup.find_all("p", class_="one_news", attrs={"data-news-dtime": True, "data-news-id": True})

        # Iterate over the found elements
        for element in news_elements:
            # Create a new temporary DataFrame to store the data
            temp_df = pd.DataFrame(columns=columns, index=[0])

            # Extract URL
            news_url = element.find("a", class_="short-text").get("href")
            temp_df['URL'] = news_url

            # Extract date and time
            news_datetime = element.get("data-news-dtime")
            news_date, news_time = news_datetime.split(" ")
            temp_df['Date'] = news_date
            temp_df['Time'] = news_time

            # Extract news title
            news_title = element.find("a", class_="short-text").get_text(strip=True)
            temp_df['News Title'] = news_title

            # Extract news ID
            news_id = element.get("data-news-id")
            temp_df['News_id'] = news_id

            # Concatenate the temporary DataFrame to the main DataFrame
            News_Data = pd.concat([News_Data, temp_df], ignore_index=True)

# Print the DataFrame
print(News_Data)

                                                    URL        Date  \
0     /football/1116047335-sutormin-o-budushhem-vern...  2024-05-06   
1     /football/1116047173-fakel-zenit-onlajn-transl...  2024-05-06   
2     /football/1116042428-chempionat-rossii-krasnod...  2024-05-06   
3     /football/1116047082-zarema-o-50-ochkax-u-lide...  2024-05-06   
4     /football/1116046899-krugovoj-krasivo-zabil-za...  2024-05-06   
...                                                 ...         ...   
8095  /football/1115806035-mostovoj-ob-abaskale-v-re...  2024-01-30   
8096  /football/1115805912-mostovoj-o-transfere-ugal...  2024-01-30   
8097  /mediafootball/1115805809-cherdanczev-sprosil-...  2024-01-30   
8098  /football/1115805810-gajich-ob-istorii-s-bolbo...  2024-01-30   
8099  /football/1115805727-bilety-na-matchi-winline-...  2024-01-30   

             Time                                         News Title  \
0     10:47:00+03  Сутормин о будущем: «Вернусь в «Зенит» по окон...   
1  

In [39]:
# Save DataFrame to Excel
News_Data.to_excel("news_data.xlsx", index=False)

# 2 - Parsing news content from collected list of URLs

In [5]:
from tqdm import tqdm

In [ ]:
# Read the Excel file into a DataFrame
df = pd.read_excel("news_data_total.xlsx")
# Extract the column containing the URLs
urls_column = df["URL"]
# Convert the column to a Python list
URL_List = urls_column.tolist()
URL_List

In [ ]:
def fetch_data_and_update_df(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find the index of the row corresponding to the URL
            index = df[df['URL'] == url].index[0]
  
            # Find the div with class 'news-item__tags-line'
            tags_line_div = soup.find('div', class_='news-item__tags-line')
            if tags_line_div:
                # Extract text from all <a> tags within the div
                all_news_tags = [tag.text for tag in tags_line_div.find_all('a')]
                df.at[index, 'All news tags'] = all_news_tags
                
            # Find the div with class 'news-item__content js-mediator-article'
            content_div = soup.find('div', class_='news-item__content js-mediator-article')
            if content_div:
                # Extract text from all <p> tags within the div
                paragraphs = content_div.find_all('p')
                news_content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
                df.at[index, 'News Content'] = news_content

            # Find the <p> element for 'Published by' and 'Source'
            p_elements = soup.find_all('p')
            for p_element in p_elements:
                if "Опубликовал" in p_element.text:
                    published_by = p_element.find('a').text
                    df.at[index, 'Published by'] = published_by
                if "Источник" in p_element.text:
                    source = p_element.find('a').get('href')
                    df.at[index, 'Source'] = source
            
            # Find the element with class 'comments-block-title'
            comments_title_element = soup.find('div', class_='comments-block-title')
            if comments_title_element:
                # Extract text from the element
                num_comments = comments_title_element.get_text(strip=True)
                df.at[index, 'Number of comments'] = num_comments
            
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")

In [ ]:
# Create additional dataframe to save original df unchanged
df_add=df

# add new columns to additional dataframe
add_columns = ['All news tags', 'News COntent', 'Published by', 'Source', 'Number of comments']
for col in add_columns:
    df_add[col] = None
    
# Iterate through the URLs in the 'URL' column and fetch data for each URL
for url in tqdm(df_add['URL'], desc="Processing URLs"):
    fetch_data_and_update_df(url)

In [ ]:
df_add.to_excel('total total news.xlsx')